In [ ]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
api_end = "https://api.scryfall.com"
mtg_whole = pd.read_csv("mtg_modern_whole.csv")

mtg_whole.dtypes

In [ ]:
name_, id_= [],[] # this code pulls unique names and id's of cards for removing reprints.
for i,el in enumerate(mtg_whole['name']):
    if el not in name_:
        name_.append(el)
        id_.append(mtg_whole['id'][i])
len(id_)

In [ ]:
# removes reprints from data
mtg_whole = mtg_whole.query(f"id == {id_}")
# writes the cleanned dataframe to a csv. .(always save work!!!)
mtg_whole.to_csv('mtg_modern_clean.csv')

In [ ]:
mtg_modern = pd.read_csv('mtg_modern_clean.csv')

In [ ]:
mtg_modern['name'].where(mtg_modern['text'].isna()).dropna(how='all').shape # cards with no rules text. 

In [ ]:
# pulls only usable data for clustering.
mtg_whole_2 = mtg_modern[['name', 'names', 'power', 'toughness', 'text', 'type', 'types', 'subtypes', 'supertypes', 'rarity', 'colorIdentity', 'colors', 'convertedManaCost', 'manaCost', 'faceConvertedManaCost',  'side', ]] # filters that would matter.
#'id', 'borderColor', 'enhrecRank', 'tcgplayerProductId', 'uuid', 'watermark', 'variations', 'originalText', 'originalType', 'scryfallIllustrationId', 'scryfallOracleId'])
mtg_whole_2.head(3)

In [ ]:
names = ['Green','Blue','Black','Red','White']
start_ = ['G','U','B','R','W']

for i,el in enumerate(names): # creates columns with cost of colors
    mtg_whole_2[f'manaCost_{el}'] = mtg_whole_2['manaCost'].apply(lambda x: str(x).count(f'{start_[i]}'))

# bellow gennerates cost of colorless. 
mtg_whole_2['manaCost_colorless'] = mtg_whole_2['convertedManaCost'].astype('int') - sum([mtg_whole_2['manaCost_Green'],mtg_whole_2['manaCost_Red'],mtg_whole_2['manaCost_Black'],mtg_whole_2['manaCost_Blue'], mtg_whole_2['manaCost_White']])

mtg_whole_2.drop(columns = ['manaCost','colors'], inplace = True)
mtg_whole_2.head(2)

In [ ]:
# does the same as above but to color identity

for i,el in enumerate(names): # creates columns with color identityof colors
    mtg_whole_2[f'colorIdentity_{el}'] = mtg_whole_2['colorIdentity'].apply(lambda x: str(x).count(f'{start_[i]}'))
    
mtg_whole_2['colorIdentity_colorless'] = [1 if  mtg_whole_2['colorIdentity'].query("colorIdentity == None")]

# bellow gennerates cost of colorless. 
# mtg_whole_2['manaCost_colorless'] = mtg_whole_2['convertedManaCost'].astype('int') - sum([mtg_whole_2['manaCost_Green'],mtg_whole_2['manaCost_Red'],mtg_whole_2['manaCost_Black'],mtg_whole_2['manaCost_Blue'], mtg_whole_2['manaCost_White']])

mtg_whole_2.drop(columns = ['colorIdentity'], inplace = True)
mtg_whole_2.head(2)

In [ ]:
# separates data by distinct card type for clustering within these.
creatures = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Creature')).dropna(how='all') # add artifact/enchanntment creatures.
spells = mtg_whole_2.where(mtg_whole_2['types'].str.endswith(('Sorcery', 'Instant'))).drop(columns = [ 'power', 'toughness']).dropna(how='all') # add trybal spells.

artifacts = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Artifact')).drop(columns = ['power', 'toughness']).dropna(how='all') # filter out artefact lands

enchantments = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Enchantment' )).drop(columns = [ 'power', 'toughness']).dropna(how='all')

planeswalkers = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Planeswalker')).drop(columns = [ 'power', 'toughness']).dropna(how='all')

In [ ]:
# mtg_whole_2['types'].value_counts()
# spells['types'].value_counts()
comanders = creatures.where(creatures['type'].str.startswith('Legendary Creature')).dropna(how = 'all')

In [ ]:
comanders.count()